In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

function, gradient estimates

In [ ]:
# Dimension of strongly convex quadratic form
dim = 2

def defineA(L=10,mu=1):
  # for a matrix with many low eigenvalues
  A = mu* np.ones(dim)
  # # for a matrix with many high eigenvalues
  # A = L* np.ones(dim)
  # # for a matrix with evenly spaced eigenvalues
  # A= np.linspace(mu, L, dim)
  A[0]=mu
  A[-1]=L
  return(A)

def f(x):
  return np.matmul (x**2, A)/2.0

def df(x):
  return np.multiply(x,A)

# Isotropic noise
def g(x,sigma=0):
  g = df(x)
  noise = np.random.normal(scale = sigma/np.sqrt(dim), size = [x.shape[0],dim])
  intensity = np.linalg.norm(g, axis = 1, ord=2)
  return g + noise*intensity[:,None]

# Noise colinear to the gradient direction
def g_colin(x,sigma=0):
  g = df(x)
  noise = np.random.normal(scale = sigma, size = x.shape[0])
  return g + g*noise[:,None]

# Noise orthogonal to the gradient direction
# Dimension 2 only!
def g_orthogonal(x, sigma=0):
  g = df(x)
  noise = np.random.normal(scale = sigma, size = x.shape[0])
  noise = g*noise[:,None]
  noise = np.array([noise[:,1], -noise[:,0]]).T
  return g + noise

# Noise in the fixed direction (1,...,1)
def g_diagonal(x, sigma=0):
  g = df(x) 
  intensity = np.linalg.norm(g, axis =1, ord=2)
  noise = np.random.normal(scale = sigma/np.sqrt(dim), size = x.shape[0])
  return g + np.multiply(noise, intensity)[:,None]


Small block for test purposes

In [ ]:
A = defineA()
x = np.ones(shape=[10000, dim])
gf= g_colin(x,20)
truegrad = df(np.ones(dim))
print (np.average(gf, axis =0))
norms = np.linalg.norm(gf - truegrad, ord =2, axis = 1)
print(np.sqrt(np.average(norms**2)/ np.linalg.norm(truegrad)**2) )

[0.93151835 9.31518349]
19.908381903326237


Running simulations

In [ ]:
def run_plot(L,sigma):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current state: sigma = "+str(sigma)+", L = "+str(L)+ ", time = "+str(current_time))
    mu = 1
    A = defineA(L,mu=1)

    eta = 1/(L*(1+sigma**2))
    gamma = 1
    
    anisotropy = 1.0- np.sqrt(mu/L)
    alpha =  anisotropy / (anisotropy + sigma**2) *eta
    rho = (np.sqrt(L)*(1+ sigma**2) - np.sqrt(mu)) / (np.sqrt(L)*(1+ sigma**2) + np.sqrt(mu))
    #rhonest = rho

    x = np.ones(shape = [noofruns,dim])
    v = np.zeros(shape = [noofruns,dim])
    xgd = np.ones(shape = [noofruns,dim])
    val = np.zeros(noofruns)
    valgd = np.zeros(noofruns)
    # xnest = np.ones(shape = [noofruns,dim])
    # xnestprev = np.ones(shape = [noofruns,dim])
    # ynest = np.ones(shape = [noofruns,dim])

    mean = np.zeros(T)
    meangd = np.zeros(T)
    std = np.zeros(T)
    stdgd = np.zeros(T)
          
    for n in range(T):
      if n%50000 ==0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("   Time is " +str (current_time) + ", steps = ",n)
      for i in range(effectiveruns):
        xcurrent = x[range(int(i*batchsize), int((i+1)*batchsize))]
        vcurrent = v[range(int(i*batchsize), int((i+1)*batchsize))]
        val[range(int(i*batchsize), int((i+1)*batchsize))] = f(xcurrent)
        xprime = xcurrent + alpha*vcurrent
        grad = g(xprime,sigma)
        #grad = g_colin(xprime,sigma)
        v[range(int(i*batchsize), int((i+1)*batchsize))] = rho*(vcurrent-gamma*grad)
        x[range(int(i*batchsize), int((i+1)*batchsize))] = xprime - eta*grad

        xcurrent = xgd[range(int(i*batchsize), int((i+1)*batchsize))]
        valgd[range(int(i*batchsize), int((i+1)*batchsize))] = f(xcurrent)
        gdgrad = g(xgd, sigma)
        #gdgrad = g_colin(xcurrent,sigma)
        xgd[range(int(i*batchsize), int((i+1)*batchsize))] = xcurrent - eta * gdgrad

        # Not modified appropriately
        # valnest[range(int(i*batchsize), int((i+1)*batchsize)),n] = f(xnest)
        # nestgrad = g_colin(ynest, sigma)
        # xnestprev = xnest
        # xnest = ynest - eta* nestgrad
        # ynest = xnest + rhonest * (xnest - xnestprev)

      mean[n]=np.mean(val)
      std[n]=np.std(val)
      meangd[n]=np.mean(valgd)
      stdgd[n]=np.std(valgd)
    return((mean, meangd))        
    #return((mean, std, meangd, stdgd))        

Perform experiments, create plots

In [ ]:
globalnoofruns = 1000 #for averaging over randomness
golablbatchsize= 1000

T = 500000 #number of steps the algorithm takes
sigmas = [0, 10, 50]
Ls = [1e4]

means = np.zeros(shape = [len(Ls), len(sigmas), T])
meansgd = np.zeros(shape = [len(Ls), len(sigmas), T])

for i in range(len(Ls)):
  for j in range (len(sigmas)):
    L = Ls[i]
    sigma = sigmas[j]
    if sigma ==0:
      noofruns = 1
      batchsize=1
    else:
      noofruns = globalnoofruns
      batchsize=golablbatchsize
    effectiveruns = int(noofruns/batchsize)
    means[i,j,:], meansgd[i,j,:] = run_plot(L, sigma)

for i in range(len(Ls)):
  plt.figure()
  plt.title("mu = 1, L = "+str(Ls[i]))
  plt.ylim(bottom = 1e-9, top = 2e1)
  plt.semilogy(means[i,0, :], color = 'red', label = "AGNES, sigma = "+str(sigmas[0]))
  plt.semilogy(meansgd[i,0, :], color = 'blue', label = "GD, sigma = "+str(sigmas[0]))
  plt.semilogy(means[i,1, :], color = 'red', linestyle = '--', label = "AGNES, sigma = "+str(sigmas[1]))
  plt.semilogy(meansgd[i,1, :],color = 'blue', linestyle = '--', label = "GD, sigma = "+str(sigmas[1]))
  plt.semilogy(means[i,2, :], color ='red', linestyle = ':', label = "AGNES, sigma = "+str(sigmas[2]))
  plt.semilogy(meansgd[i,2, :],color = 'blue', linestyle = ':', label = "GD, sigma = "+str(sigmas[2]))
  plt.legend()

Plotting only

In [ ]:
#means10 = means[0,:,:]
#meansgd10 = meansgd[0,:,:]
#means500 = means[0,:,:] 
#meansgd500 = means10 
#means1e4 = meansgd[0,:,:]
#meansgd1e4 = meansgd[0,:,:] 

for i in range(len(Ls)):
  plt.figure()
  plt.ylim(bottom = 1e-9, top = 2e1)
  plt.title("mu = 1, L = "+str(Ls[i]))
  plt.semilogy(means[i,0, :], color = 'red', label = "AGNES, sigma = "+str(sigmas[0]))
  plt.semilogy(meansgd[i,0, :], color = 'blue', label = "GD, sigma = "+str(sigmas[0]))
  plt.semilogy(means[i,1, :], color = 'red', linestyle = '--', label = "AGNES, sigma = "+str(sigmas[1]))
  plt.semilogy(meansgd[i,1, :],color = 'blue', linestyle = '--', label = "GD, sigma = "+str(sigmas[1]))
  plt.semilogy(means[i,2, :], color ='red', linestyle = ':', label = "AGNES, sigma = "+str(sigmas[2]))
  plt.semilogy(meansgd[i,2, :],color = 'blue', linestyle = ':', label = "GD, sigma = "+str(sigmas[2]))
  plt.legend()